In [1]:
import numpy as np
from quaos.core.paulis import PauliSum, PauliString
from quaos.core.circuits.target import find_map_to_target_pauli_sum
from quaos.core.circuits import Gate, Circuit, SWAP
from quaos.utils import get_linear_dependencies
from quaos.graph_utils import find_one_permutation, permutation_to_swaps, mapping_key, brute_force_all_permutations, find_swapped_dependent_elements
from quaos.models import ToricCode, Hadamard_Symmetric_PauliSum, SWAP_symmetric_PauliSum
from scripts.experiments.symmetries.src.pauli_symmetries import symplectic_pauli_reduction 

First we make a random Pauli Sum, and obtain a list of the linearly independent rows

In [ ]:
toric = True
hadamard_symmetry = False
specific = False
swap_symmetry = False

if toric:
    d = 2

    Nx = 2
    Ny = 8
    periodic = True
    c_x = 1.
    c_z = 1.
    c_g = 1.

    TC = ToricCode(Nx, Ny, c_x, c_z, c_g, periodic)
    H_tc = TC.hamiltonian()
    print("original toric code Hamiltonian")
    print(H_tc)
    C_reduce = symplectic_pauli_reduction(H_tc)
    H_red = H_tc #C_reduce.act(H_tc)
    # print(H_red)
    H = H_red # [:, 0:-5]  # as here there are 3 Pauli symmetries, should automate this bit
    H.remove_trivial_paulis()
    # print(H)
    H.combine_equivalent_paulis()
elif hadamard_symmetry:
    
    d = 2
    n_qubits = 5
    n_sym_q = 2
    n_paulis = 12
    H, C = Hadamard_Symmetric_PauliSum(n_paulis, n_qubits, n_sym_q)
    H.combine_equivalent_paulis()
elif swap_symmetry:
    d = 2
    n_qubits = 5
    n_paulis = 12
    H = SWAP_symmetric_PauliSum(n_paulis, n_qubits)
    C_reduce = symplectic_pauli_reduction(H)
    H = C_reduce.act(H)
    # H.combine_equivalent_paulis()
    n_paulis = H.n_paulis()
    H.weights = np.ones(n_paulis, dtype=int)  # set all weights to 1

elif specific:
    d = 2 
    dimensions=[d, d, d, d]
    # p_string = ['x1z0 x1z0 x0z1', 'x0z1 x0z1 x1z1', 'x1z1 x1z1 x1z1', 'x1z1 x1z1 x1z0']
    # p_string = ['x1z0 x0z0', 'x0z0 x1z0', 'x0z1 x0z0', 'x0z0 x0z1', 'x0z1 x0z1'] 
    p_string = ['x0z1 x0z0 x0z0 x0z0', 
                'x0z0 x0z1 x0z0 x0z0',
                'x0z0 x0z0 x0z1 x0z0', 
                'x0z0 x0z0 x0z0 x0z1',
                'x1z0 x0z0 x0z0 x0z0',
                'x0z0 x1z0 x0z0 x0z0', 
                'x0z0 x0z0 x1z0 x0z0', 
                'x0z0 x0z0 x0z0 x1z0',
                'x0z1 x0z1 x0z0 x0z0',
                'x0z1 x0z0 x0z1 x0z0',
                'x0z0 x0z1 x0z0 x0z1',
                'x0z0 x0z0 x0z1 x0z1'] 
    H = PauliSum(p_string, dimensions=dimensions, weights=[1 for _ in range(len(p_string))], standardise=False)
    # C = Circuit.from_random(len(dimensions), 100, dimensions=dimensions)
    # H = C.act(H)
    # C_reduce = symplectic_pauli_reduction(H)
    # H = C_reduce.act(H) 
else:
    # parameters
    n_paulis = 20
    n_qudits = 8
    d = 2
    n_weights = 1 # number of different weights

    dimensions = [d] * n_qudits 
    # make a random pauli sum
    H = PauliSum.from_random(n_paulis, n_qudits, dimensions, rand_weights=False)

    # this bit makes sections of H have different weights
    weights = np.empty(n_paulis, dtype=int)
    section_size = n_paulis // n_weights
    for i in range(n_weights):
        start = i * section_size
        end = (i + 1) * section_size if i < n_weights - 1 else n_paulis
        weights[start:end] = i + 1

    H.weights = weights
    # print(H.symplectic())

# obtain linearly independent rows - note this fails sometimes as there is a solver in it that
# is not that robust. The Clifford approach is more robust as it creates a simple basis where the
# dependencies can be easily read.
print("Resulting Hamiltonian:")
print(H)
independent_paulis, dependencies = get_linear_dependencies(H.symplectic(), d)

print(independent_paulis)
print(dependencies)



KeyboardInterrupt: 

We now solve for possible permutations which leave the dependencies unchanged

To do so we make a dictionary with labels of the coefficients

***Note this will only work for qubits for now, as for qudits we also need to account for the fact that a dependency can be made of $P_i + kP_j$ for some integer $k < d$***

In [15]:
cs = H.weights

graph_dict = {}

for i in independent_paulis:
    key = cs[i]
    if key in graph_dict:
        graph_dict[key].append([i])
    else:
        graph_dict[key] = [[i]]

for i in dependencies.keys():
    key = cs[i]
    dependency = dependencies[i]
    dependence_indices = [x[0] for x in dependency]
    dependence_multiplicities = [x[1] for x in dependency]  # this will be needed for qudits! always 1 for now
    if key in graph_dict:
        graph_dict[key].append(dependence_indices)
    else:
        graph_dict[key] = [dependence_indices]

print(graph_dict)

{np.complex128(1+0j): [[0], [1], [2], [5], [8], [0, 2], [1, 2], [0, 5], [1, 5]]}


We then test to see if any indexes have an automorphism that can be found via a simple swap

In [16]:

permutation = ()
permutations_found = set()
permutations_attempted = set()
found = False
i = 0
all_permutations = brute_force_all_permutations(graph_dict[1], np.ones(len(graph_dict[1]), dtype=int))
print([permutation_to_swaps(perm) for perm in all_permutations])
while not found:
    i += 1
    print(f"Attempting permutation {i}")
    permutation = find_one_permutation(graph_dict[1], np.ones(len(graph_dict[1]), dtype=int), permutations_attempted, 
                                       max_cycle_size=300)
    if permutation is None:
        raise ValueError("No valid permutation found")
    
    print(permutation_to_swaps(permutation))
    pairs = permutation_to_swaps(permutation)
    swapped_dependents = find_swapped_dependent_elements(pairs, graph_dict[1])
    print('sd = ', swapped_dependents)
    H_target = H.copy()
    # for p in pairs:
    #     H_target.swap_paulis(p[0], p[1]) 
    for p in swapped_dependents:
        H_target.swap_paulis(p[0], p[1])

    if np.array_equal(H_target.symplectic_product_matrix(), H.symplectic_product_matrix()):
        found = True
        print("Found automorphism")
        break
        # permutations_attempted.add(mapping_key(permutation, domain=sorted({x for lst in graph_dict[1] for x in lst})))
        # permutations_found.add(mapping_key(permutation, domain=sorted({x for lst in graph_dict[1] for x in lst})))

    else:
        print("Not an automorphism, trying next permutation")
        permutations_attempted.add(mapping_key(permutation, domain=sorted({x for lst in graph_dict[1] for x in lst})))

print(pairs)

[[], [(2, 5)], [(0, 1)], [(0, 1), (2, 5)], [(0, 2), (1, 5)], [(0, 2), (0, 1), (0, 5)], [(0, 5), (0, 1), (0, 2)], [(0, 5), (1, 2)]]
Attempting permutation 1
[(0, 1)]
sd =  [(0, 1), (5, 6), (7, 8)]
Not an automorphism, trying next permutation
Attempting permutation 2
[(2, 5)]
sd =  [(2, 3), (5, 7), (6, 8)]
Not an automorphism, trying next permutation
Attempting permutation 3
[(0, 1), (2, 5)]
sd =  [(0, 1), (2, 3), (5, 8), (6, 7)]
Not an automorphism, trying next permutation
Attempting permutation 4
[(0, 2), (1, 5)]
sd =  [(0, 2), (1, 3), (6, 7)]
Not an automorphism, trying next permutation
Attempting permutation 5
[(0, 2), (0, 1), (0, 5)]
sd =  [(0, 3), (1, 0), (2, 0), (5, 7)]
Not an automorphism, trying next permutation
Attempting permutation 6
[(0, 5), (0, 1), (0, 2)]
sd =  [(0, 2), (1, 0), (3, 0), (7, 5)]
Not an automorphism, trying next permutation
Attempting permutation 7
[(0, 5), (1, 2)]
sd =  [(0, 3), (1, 2), (5, 8)]
Not an automorphism, trying next permutation
Attempting permutat

ValueError: No valid permutation found

Now we choose a target - for this example, performing all of these swaps

We then find the symplectic which maps the original H to H_target

In [13]:
# found = False
# # make target
# H_target = H.copy()
# for p in pairs:  # This is inefficient - could be done better by checking only the commutation of Paulis to be swapped
#     H_target.swap_paulis(p[0], p[1])
#     if np.array_equal(H_target.symplectic_product_matrix(), H.symplectic_product_matrix()):
#         found = True
#         print("Found automorphism")
#         break

# if not found:
#     print("No automorphism found - there are only non-Pauli symmetries if the rank of H is < 2n")
#     H_target = H.copy()


# find F
# It may be here that we need to use only the paulis that are linearly independent
H_indep = H[independent_paulis]
H_t_indep = H_target[independent_paulis]

assert np.all(H_indep.symplectic_product_matrix() == H_t_indep.symplectic_product_matrix())

# print(H_indep)
# print(H_t_indep)


F, _, _, _ = find_map_to_target_pauli_sum(H_indep, H_t_indep)

# print(F)
# check F
# H_target.swap_paulis(3, 4)
# H_target.swap_paulis(7, 9)


print(H.symplectic())
print((H_target.symplectic() @ F) % d)
# print(H.symplectic() @ F % d)
print(np.array_equal(H_target.symplectic(), H.symplectic() @ F % d))
print(F)


[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1]]
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1]]
True
[[0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]]
